In [1]:
import pandas as pd

In [2]:
# индексы выборок в массиве выборок
TRAIN, TEST = 0, 1
# файлы с выборками
files = ['train_prepared.csv', 'test_prepared.csv']

dfs = []
for file in files:
    dfs.append(pd.read_csv(file))
    dfs[-1]['Sex'] = dfs[-1]['Sex'].map(lambda x: 1 if x == 'female' else 0)
    dfs[-1]['Sport'] = dfs[-1]['Sport'].map(lambda x: 1 if x == '+' else 0)
    dfs[-1] = dfs[-1].drop(["Unnamed: 0"], axis="columns")

dfs[TRAIN]

,Longevity,Education,Sex,Age,Pet,Children,Activity,Sport,Family,IsAlone,SocialStatus,AgeInterval
0,0,3,1,73.0,1,0,725,0,1,False,1,3
1,1,1,0,75.0,1,0,7128,1,1,False,6,4
2,1,3,0,73.0,0,0,793,0,0,True,1,3
3,1,1,0,74.0,1,0,5310,1,1,False,5,3
4,0,3,1,74.0,0,0,805,0,0,True,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
851,0,3,0,71.0,1,1,1525,1,2,False,1,2
852,1,1,0,72.0,0,1,3940,1,1,False,5,2
853,0,2,0,76.0,1,0,2600,0,1,False,4,4
854,1,3,0,72.0,0,1,935,0,1,False,1,2


In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# функция создания пайплайна для модели (со стандартным масштабированием) и проверки точности модели
# fit на тренировочной и проверка на тестовой выборках
def check_model(model):
    clf = make_pipeline(StandardScaler(), model)
    clf.fit(dfs[TRAIN].drop(["Longevity"], axis="columns"), dfs[TRAIN]["Longevity"])
    return clf.score(dfs[TEST].drop(["Longevity", "Id"], axis="columns"),
                          dfs[TEST]["Longevity"])


In [4]:
# Алгоритм SVM ищет объекты данных (вектора), которые расположены ближе
# всего к линии разделения. Эти точки называются опорными векторами. 
# Затем, алгоритм вычисляет расстояние между опорными векторами и 
# разделяющей плоскостью (это расстояние называется зазором). Основная
# цель алгоритма — максимизировать расстояние зазора. Лучшей
# гиперплоскостью считается такая гиперплоскость, для которой этот
# зазор является максимально большим.

from sklearn.svm import SVC, LinearSVC

acc_svc = check_model(SVC(gamma='auto'))
acc_linear_svc = check_model(LinearSVC(random_state=0, tol=1e-5, max_iter=1e6))
acc_svc, acc_linear_svc

(0.9175946547884187, 0.9131403118040089)

In [5]:
# kNN (k Nearest Neighbor) – алгоритм k-ближайших соседей использует
# весь набор данных в качестве обучающего набора, а не разделяет набор
# данных на обучающий набор и набор тестов. Когда для нового экземпляра
# данных требуется результат, алгоритм KNN просматривает весь набор
# данных, чтобы найти k-ближайших экземпляров для нового экземпляра, 
# или k экземпляров, наиболее похожих на новую запись, а затем выводит
# среднее значение результаты (для регрессии) или наиболее близкий
# класс (для задачи классификации).

from sklearn.neighbors import KNeighborsClassifier

acc_knn = check_model(KNeighborsClassifier(n_neighbors=50))
acc_knn

0.8975501113585747

In [6]:
# Наи́вный ба́йесовский классифика́тор — простой вероятностный 
# классификатор, основанный на применении теоремы Байеса со строгими
# (наивными) предположениями о независимости.
from sklearn.naive_bayes import GaussianNB

acc_gaussian = check_model(GaussianNB())
acc_gaussian

0.799554565701559

In [7]:
# Логистическая регрессия — это статистическая модель, используемая
# для прогнозирования вероятности возникновения некоторого события
# путём его сравнения с логистической кривой. Эта регреcсия выдаёт
# ответ в виде вероятности бинарного события (1 или 0).

#  Стохастический градиентный спуск - это метод итерации для
# оптимизации целевой функции с подходящими свойствами гладкости
# (например, дифференцируемость или субдифференцируемость). Его можно
# расценивать как стохастическую аппроксимацию оптимизации методом 
# градиентного спуска, поскольку он заменяет реальный градиент 
# (вычисленный из полного набора данных[en]) путём оценки такового
# (вычисленного из случайно выбранного подмножества данных)

from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier

acc_log = check_model(LogisticRegression(random_state=0))
acc_perceptron = check_model(Perceptron(tol=1e-3, random_state=0))
acc_sgd = check_model(SGDClassifier(max_iter=1e6, tol=1e-3))
acc_log, acc_perceptron, acc_sgd

(0.9042316258351893, 0.734966592427617, 0.8596881959910914)

In [8]:
# Деревья решений.
# Идея: Дерево строится «сверху вниз» от корня. 
# Начинается процесс с определения, какой атрибут следует выбрать
# для проверки в корне дерева. Для этого каждый атрибут исследуется
# на предмет, как хорошо он классифицирует набор данных (разделяет
# на классы по целевому атрибуту). При этом выбирается тот из
# атрибутов, который порождает наибольший количественный критерий 
# оценки. Когда атрибут выбран, для каждого его значения создается
# ветка дерева, набор данных разделяется в соответствии со значением
# к каждой ветке, процесс повторяется рекурсивно для каждой ветки.
# Также следует проверять критерий остановки.

# В деревьях классификации часто используются перекрестная энтропия,
# энтропия Шеннона и коэффициент Джини. В деревьях регрессии
# минимизируется сумма функций потерь. Мы выполняем эту процедуру
# рекурсивно для каждого узла и завершаем работу, когда выполняем 
# критерии остановки.

# В качестве критерия остановки могут быть выбраны: минимальное
# количество уровней дерева от листа до вершины, минимальное
# значение критерия оценки в узле и пр.

from sklearn.tree import DecisionTreeClassifier

acc_decision_tree = check_model(DecisionTreeClassifier(random_state=0))
acc_decision_tree

0.8530066815144766

In [9]:
# Random forest (с англ. — «случайный лес») — алгоритм машинного 
# обучения, заключающийся в использовании комитета (ансамбля) решающих
# деревьев. Алгоритм сочетает в себе две основные идеи: метод бэггинга
# Бреймана, и метод случайных подпространств, предложенный Тин Кам Хо.
# Алгоритм применяется для задач классификации, регрессии и
# кластеризации. Основная идея заключается в использовании большого 
# ансамбля решающих деревьев, каждое из которых само по себе даёт
# очень невысокое качество классификации, но за счёт их большого
# количества результат получается хорошим.

from sklearn.ensemble import RandomForestClassifier

acc_random_forest = check_model(RandomForestClassifier(max_depth=4, random_state=0))
acc_random_forest

0.8574610244988864

In [10]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression',
              'Random Forest', 'Naive Bayes', 'Perceptron',
              'Stochastic Gradient Decent', 'Linear SVC',
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log,
              acc_random_forest, acc_gaussian, acc_perceptron,
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)


,Model,Score
0,Support Vector Machines,0.917595
7,Linear SVC,0.913140
2,Logistic Regression,0.904232
1,KNN,0.897550
6,Stochastic Gradient Decent,0.859688
3,Random Forest,0.857461
8,Decision Tree,0.853007
4,Naive Bayes,0.799555
5,Perceptron,0.734967
